---
# UKB_AHA_event_.csv 를 이용해서 label달기
### AHA는 eid이용
1. csv파일 총 image수 확인
2. csv파일에 있는 image들 hq들만 있는지 확인(lq, hq 다 있어야함)  
--> HL, LQ에는 없음  
--> ALL에는 
3. label 달아서(다정상:0, 하나라도 비정상:1) csv파일로 저장

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

---
#### 0. UKB_fundus_ALL_nolabel 의 이미지명 (all_name)과 eid (all_eid)를 저장

In [71]:
path = "/home/guest1/UKB_fundus_ALL_nolabel"
all = []
all_eid = set()
for img in tqdm(os.listdir(path)):
    all.append(img)
    eid = img.split("_")[0]
    all_eid.add(eid)
all_ar = np.array(all)
all_eid_ar = np.array(list(all_eid))

np.save("./all_name",all_ar)
np.save("./all_eid",all_eid_ar)

print("all file 개수: ", len(all))
print("name unique 개수: ", len(set(all)))
print("eid unique 개수: ", len(all_eid))

100%|██████████| 175796/175796 [00:00<00:00, 1724139.07it/s]

all file 개수:  175796
name unique 개수:  175796
eid unique 개수:  85711


In [72]:
print("total length of all_eid_array: ",len(all_eid_ar))
print("example: ", all_eid_ar[0:5])

total length of all_eid_array:  85711
example:  ['3323759' '3591873' '2115657' '4661417' '1682695']


---
#### 0. highq, lowq이미지 numpy 각각 만들기

In [2]:
path = "/home/guest1/ellen_data/UKB_quality_data2_combined"

high =[]
for img in os.listdir(path+"/"+"high_Q"):
    # print(img)
    high +=[img]

print(len(high))

high_ar = np.array(high)
np.save("./high_name",high_ar)
    

8020


In [3]:
path = "/home/guest1/ellen_data/UKB_quality_data2_combined"

low =[]
for img in os.listdir(path+"/"+"low_Q"):
    # print(img)
    low +=[img]

print(len(low))
low_ar = np.array(low)
np.save("./low_name",low_ar)
    

2526


#### 0-1. high, low 에서 eid만 뽑아오기

In [4]:
high_eid=[]
count =0
for imgname in high:
   eid=imgname.split("_")[0]
   high_eid +=[eid]
print(len(high_eid))

high_eid_ar = np.array(high_eid)
np.save("./high_eid",high_eid_ar)


8020


In [5]:
low_eid=[]
count =0
for imgname in low:
   eid=imgname.split("_")[0]
   low_eid +=[eid]
print(len(low_eid))

low_eid_ar = np.array(low_eid)
np.save("./low_eid",low_eid_ar)


2526


#### 0-2. high, low 에서 eid 쭝복된 것 있는지  
--> 중복된 것 존재

In [6]:
high_eid_set = set(high_eid)
print(len(high_eid_set) == len(high_eid))
print(len(high_eid))
print(len(high_eid_set))

False
8020
7588


In [7]:
low_eid_set = set(low_eid)
print(len(low_eid_set) == len(low_eid))
print(len(low_eid))
print(len(low_eid_set))

False
2526
2443


---
#### 1. csv 파일 총 image 수 확인   
--> csv파일의 eid는 중복 없음

In [8]:
aha_df = pd.read_csv("/home/guest1/UKB_AHA_event.csv")

eid를 string type으로 바꿔주기, UKB hq,lq에 있는 type과 맞춰주기 위해서

In [92]:
df_len = len(aha_df)
aha_df['eid']=aha_df['eid'].astype(str)

1-1. 중복된 image명 있는지 확인


In [85]:
aha_df.drop_duplicates(["eid"])
print(len(aha_df))
if df_len ==len(aha_df):
    print(">>> image 명(eid) 중복 없음")
else: 
    print(">>> image 명(eid) 중복 있음!!!")

67288
>>> image 명(eid) 중복 없음


---
#### 2. csv파일에 있는 이미지들 어디에 있는 것인지 확인 - h,l
hl_csv의 열 (1) eid, (2) high, (3) low (4) AHA_event  
아래 block 실행하는데 오래 걸림

In [18]:
import time
from tqdm import tqdm

In [95]:
hl_csv = pd.DataFrame(columns=['eid', "high", "low", "AHA_event"])
for i in tqdm(range(df_len)):
    h=l=0                           
    eid = aha_df.iloc[i,0]
    disease = aha_df.iloc[i,1]
    hl_csv.loc[i] =[eid, eid in high_eid, eid in low_eid, disease]

100%|██████████| 67288/67288 [03:35<00:00, 312.19it/s]


In [96]:
hl_csv.to_csv("./AHA_hqlq.cvs")

In [97]:
print(len(hl_csv))
print(len(hl_csv)==df_len)

67288
True


UKB_quality_data & UKB_quality_data2 합친 것 (UKB_quality_data2_combined)에 포함되어있는 데이터  
즉, quality data 중 AHA label이 있는 것

In [99]:
print(" # of high quality data in 'eye_label': " ,len(hl_csv[hl_csv['high'] ==True]))
print(" # of low quality data in 'eye_label': " ,len(hl_csv[hl_csv['low'] ==True]))
print("===================================")
print("# of high + low quality data in 'eye_lable': ",len(hl_csv[hl_csv['high'] ==True])+len(hl_csv[hl_csv['low'] ==True]),"/",df_len )

 # of high quality data in 'eye_label':  7540
 # of low quality data in 'eye_label':  2413
# of high + low quality data in 'eye_lable':  9953 / 67288


---
#### 2-1. csv파일에 있는 이미지들 어디에 있는 것인지 확인 - all

In [94]:
count =0
for i in tqdm(range(df_len)):
    eid = aha_df.iloc[i,0]
    if eid in all_eid_ar:
        count+=1
print("file in ALL: ",count,"/",df_len)

100%|██████████| 67288/67288 [00:42<00:00, 1595.05it/s]

file in ALL:  67288 / 67288


---
#### 2-2. AHA event happend

In [54]:
eventt_df = aha_df[aha_df["all_AHA_event"]==1]
eventt_df

,eid,all_AHA_event,all_AHA_event_year
2,1000326,1,8.005476
15,1001397,1,2.696783
32,1002799,1,6.464066
60,1005431,1,5.869952
73,1006046,1,9.519507
...,...,...,...
67135,6014023,1,3.383984
67150,6014987,1,7.411362
67197,6017703,1,7.214237
67261,6022531,1,8.104038
